In [ ]:
import cv2
import os
import numpy as np

def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    if face_classifier.empty():
        print("Error loading haarcascade_frontalface_default.xml")
        return

    # Load recognizer
    clf = None
    if os.path.exists("classifier.xml"):
        clf = cv2.face.LBPHFaceRecognizer_create()
        clf.read("classifier.xml")
        print("Loaded existing classifier.")
    else:
        print("classifier.xml not found. Skipping recognition check.")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.1, 4)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face, (x, y, w, h)
        return None

    if not os.path.exists("facedata"):
        os.makedirs("facedata")

    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    if not cap.isOpened():
        print("Could not open webcam")
        return

    # Recognition attempt before collecting data
    recognized_id = None
    if clf is not None:
        attempts = 0
        while attempts < 30:
            ret, frame = cap.read()
            if not ret:
                break

            result = face_cropped(frame)
            if result is not None:
                face, (x, y, w, h) = result
                try:
                    face_gray = cv2.cvtColor(cv2.resize(face, (200, 200)), cv2.COLOR_BGR2GRAY)
                    pred_id, confidence = clf.predict(face_gray)
                    confidence_pct = int(100 * (1 - confidence / 300))
                    if confidence_pct > 75:
                        recognized_id = pred_id
                        print(f"User already exists with ID: {recognized_id} (confidence: {confidence_pct}%)")
                        break
                except Exception as e:
                    print("Recognition error:", e)

            attempts += 1
            cv2.imshow("Checking for Existing User", frame)
            if cv2.waitKey(1) == 13:
                break

        if recognized_id is not None:
            print("User already exists. Dataset will not be generated.")
            cap.release()
            cv2.destroyAllWindows()
            return

    # New user – generate ID and capture
    existing_ids = [int(f.split(".")[1]) for f in os.listdir("facedata") if f.startswith("user.") and f.endswith(".jpg")]
    new_id = max(existing_ids, default=0) + 1

    img_id = 0
    print(f"Starting face capture for new ID: {new_id}. Press Enter or wait for 500 samples.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        result = face_cropped(frame)
        if result is not None:
            cropped_face, _ = result
            img_id += 1
            face = cv2.resize(cropped_face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            file_name_path = f"facedata/user.{new_id}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)

            cv2.putText(face, str(img_id), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped Face", face)

        if cv2.waitKey(1) == 13 or img_id == 400:
            break

    cap.release()
    cv2.destroyAllWindows()

    if img_id > 0:
        print(f"Samples have been collected for new ID: {new_id}")
    else:
        print("No samples collected.")

if __name__ == "__main__":
    generate_dataset()


In [37]:
from PIL import Image

def Train_Classifier(facedata_dir):
    path = [os.path.join(facedata_dir, f) for f in os.listdir(facedata_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')  # Grayscale
        ImageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])  # user.ID.imgid.jpg
        faces.append(ImageNp)
        ids.append(id)

    ids = np.array(ids)

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    print("Classifier Updated")

if __name__ == "__main__":
    Train_Classifier("facedata")

Classifier Updated


In [2]:
import cv2
import os
import numpy as np
def DrawBoundary(img, classifier, scaleFactor, minNeighbors, clf):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)
    coords = []

    for (x, y, w, h) in features:
        roi_gray = gray_image[y:y + h, x:x + w]
        id_, pred = clf.predict(roi_gray)
        confidence = int(100 * (1 - pred / 300))

        if confidence > 78:
            label = f"User {id_} and {confidence}"
            color = (0, 255, 0)
        else:
            label = "Unknown"
            color = (0, 0, 255)

        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        coords = [x, y, w, h]

    return coords

def recognize(img, clf, face_cascade):
    coords = DrawBoundary(img, face_cascade, 1.2, 10, clf)
    return img

if __name__ == "__main__":
    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    if face_cascade.empty():
        print("Error loading haarcascade_frontalface_default.xml")
        exit()

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture = cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        if not ret:
            print("Failed to capture frame")
            break

        img = recognize(img, clf, face_cascade)
        cv2.imshow("Face Recognition", img)

        if cv2.waitKey(1) == 13:  # Press Enter to exit
            break

    video_capture.release()
    cv2.destroyAllWindows()
